In [30]:
!mkdir static template data

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
!tar -xvf /content/ngrok-stable-linux-amd64.tgz
#https://dashboard.ngrok.com/get-started/your-authtoken link to get auth token.
!./ngrok authtoken "<YOUR-AUTH-TOKEN>"

In [ ]:
!pip install flask_ngrok

In [32]:
from flask import Flask, url_for, render_template, request, redirect, flash
from flask_ngrok import run_with_ngrok
from werkzeug.utils import secure_filename
import os
from datetime import datetime

import tensorflow as tf
import tensorflow_hub as hub

# For drawing onto the image.
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

# For closing the application.
import os


In [33]:
def resize_image(filename, new_width=256, new_height=256):
    pil_image = Image.open(filename)
    pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)
    pil_image_rgb = pil_image.convert("RGB")
    pil_image_rgb.save(filename, format="JPEG", quality=100)
    print("Image Saved to %s." % filename)

    return filename


def load_img(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    return img


def run_detector(filename):
    
    module_handle = "https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1"
    detector = hub.load(module_handle).signatures['default']
    
    resize_image(filename)
    
    img = load_img(filename)

    converted_img = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
    result = detector(converted_img)

    result = {key: value.numpy() for key, value in result.items()}


    classes_detected = [x.decode('ascii') for x in result["detection_class_entities"]]
    classes_detected = list(dict.fromkeys(classes_detected))


    os.remove(filename)

    return classes_detected

In [36]:
UPLOAD_FOLDER = os.getcwd() + '/data/'
ALLOWED_EXTENSIONS = {'jpg', 'png', 'jpeg'}

app = Flask(__name__, template_folder='template')
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
run_with_ngrok(app)
now = datetime.now()


@app.route("/", methods=['GET', 'POST'])
def index_upload():
    return render_template('form.html')
    
def allowed_filename(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route("/results", methods=['POST'])
def results():
    if request.method == 'POST':    
        if 'file' not in request.files:
            flash('No files!')
            return redirect(url_for("index_upload"))
        f = request.files['file']
        if f.filename == '':
            flash('No selected file!')
            return redirect(url_for("index_upload"))
        if f and allowed_filename(f.filename):
            file = secure_filename(f.filename)
            filename = os.path.join(app.config['UPLOAD_FOLDER'], file)
            f.save(filename)
            
            result = run_detector(filename)
            
            return render_template('results.html', value=result)

In [ ]:
app.run()